# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 40 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set
- 20 points - Evaluate on Github Typo Corpus (for masters only)


Remarks: 
- Use Python 3 or greater
- Max is 80 points for bachelors, 100 points for masters

## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html).

[N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf)

You may also wnat to implement:
- spell-checking for a concrete language - Russian, Tatar, Ukranian, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

Important - your project should not be a mere code copy-paste from somewhere. Implement yourself, analyze why it was suggested this way, and think of improvements/customization.

Your solution should be able to perform 4 corrections per second (3-5 words in an example) on a typical cpu.

In [28]:
import nltk
import requests

nltk.download('brown')
nltk.download('punkt')
nltk.download('treebank')

import re
import itertools
import random
from collections import Counter, defaultdict
from typing import List, Tuple, Set, Generator, Iterator

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [29]:
class Corrector:
    def __init__(self, ngram_dict_path: str):
        self.model = defaultdict(lambda: defaultdict(lambda: 0))
        self.nd_path = ngram_dict_path # n-gram dictionary file path
    
    def load_nd(self):
        "Load n-gram dictionary to self.model from file"
        
        with open(self.nd_path, "r") as f:
            for s in f:
                splitted = s.lower().split()
                w_key, w_value = tuple(splitted[1:-1]), tuple(splitted[-1:])
                count = int(splitted[0])
                self.model[w_key][w_value] += count
        
        for w_key in self.model:
            total_count = float(sum(self.model[w_key].values()))
            for w_value in self.model[w_key]:
                self.model[w_key][w_value] /= total_count

In [30]:
class NorvigCorrector(Corrector):
    def __init__(self, datafile: str, ngram_dict_path: str=None):
        super().__init__(ngram_dict_path)
        self.words = Counter(self.tokens(open(datafile).read()))

    def tokens(self, text: str) -> List:
        return re.findall(r'\w+', text.lower())

    def P(self, word: str, N=None) -> float:
        "Probability of `word`."
        if not N:
            N = sum(self.words.values())
        return self.words[word] / N

    def correction(self, word: str) -> str:
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word: str) -> Set:
        "Generate possible spelling corrections for word."
        return (self.known([word]) or
                self.known(self.edits1(word)) or
                self.known(self.edits2(word)) or
                [word])

    def known(self, words: Iterator) -> Set: 
        "The subset of `words` that appear in the dictionary of self.words."
        return set(w for w in words if w in self.words)

    def edits1(self, word: str) -> Set:
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word: str) -> Generator: 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

    def correct(self, string: str) -> str:
        """
        Return fixed string.
        """
        tokens = re.findall(r"\w+", string)
        words = list(map(self.correction, [token.lower() for token in tokens]))
        
        str_arr = []
        start_idx = 0
        next_start_idx = 0
        for idx, token in enumerate(tokens):
            # find index of current token
            next_start_idx = string.find(token, start_idx)
            # add everything between tokens also (e.g. punctuation)
            str_arr.append(string[start_idx:next_start_idx])
            to_add = words[idx]
            # keep word if uppercase
            if token.istitle():
                to_add = to_add.capitalize()
            elif token.isupper():
                to_add = to_add.upper()
            str_arr.append(to_add)
            # set start index to the next symbol after the word
            start_idx = next_start_idx + len(token)
        # add the rest after the last token
        str_arr.append(string[start_idx:])

        return ''.join(str_arr)
    
    def accuracy(self, 
                 source_sents: List[str],
                 spoiled_sents: List[str]
                 ) -> float:
        """
        Count accuracy for the given list of strings.
        """
        if len(source_sents) != len(spoiled_sents):
            raise ValueError("Lists of different lengths!")
        
        total_count = 0
        total_errors = 0
        for idx in range(len(source_sents)):
            corrected_sent = self.correct(spoiled_sents[idx])
            errors_count, tokens_count = self._error_counter(
                source_sents[idx], corrected_sent
                )
            total_errors += errors_count
            total_count += tokens_count
        
        return total_errors / total_count

    
    def _error_counter(self, source: str, corrected: str):
        """
        Compares two strings by tokens and counts number of errors.
        """
        source = re.findall(r"\w+", source)
        corrected = re.findall(r"\w+", corrected)
        if len(source) != len(corrected):
            raise ValueError("String tokenized to different lengths.")

        return sum([s == c for s, c in zip(source, corrected)]), len(source)

In [31]:
r = requests.get("https://norvig.com/big.txt")
big_data_path = "big.txt"
with open(big_data_path, "w") as f:
    f.write(r.text)

In [32]:
norvig = NorvigCorrector(big_data_path)
print(norvig.candidates('fyck'))

{'fuck'}


In [33]:
r = requests.get("https://raw.githubusercontent.com/aalexren/iu-nlp/master/assignment02/w2_.txt")
bigrams_path = "w2.txt"
with open(bigrams_path, "w") as f:
    f.write(r.text)

In [34]:
class BlueberryCorrector(NorvigCorrector):
    def __init__(self, datafile: str, ngram_dict_path: str):
        super().__init__(datafile=datafile,
                         ngram_dict_path=ngram_dict_path)

    def correct(self, string: str) -> str:
        """
        Return fixed string.
        """
        tokens = re.findall(r"\w+", string)
        words = self._correct_spelling(string)
        
        str_arr = []
        start_idx = 0
        next_start_idx = 0
        for idx, token in enumerate(tokens):
            # find index of current token
            next_start_idx = string.find(token, start_idx)
            # add everything between tokens also (e.g. punctuation)
            str_arr.append(string[start_idx:next_start_idx])
            to_add = words[idx]
            # keep word if uppercase
            if token.istitle():
                to_add = to_add.capitalize()
            elif token.isupper():
                to_add = to_add.upper()
            str_arr.append(to_add)
            # set start index to the next symbol after the word
            start_idx = next_start_idx + len(token)
        # add the rest after the last token
        str_arr.append(string[start_idx:])

        return ''.join(str_arr)

    def _correct_spelling(self, string: str) -> List[str]:
        """
        Correct spelling for tokens and returns list of them.
        """
        words = self.tokens(string)
        if len(words) == 1:
            return words
        
        # ws_weights = [self.candidates(word) for word in words[:2]]
        ws = [self.candidates(word) for word in words[:2]]
        bigrams = itertools.product(*ws)
        max_prob = max(bigrams, 
                        key=lambda x: self.model[x[:-1]][x[-1:]])
        words[0] = max_prob[0]
        for idx in range(1, len(words)):
            bigram = words[idx - 1], words[idx]
            ws = [self.candidates(word) for word in bigram]
            bigrams = itertools.product(*ws)
            max_prob = max(bigrams, 
                           key=lambda x: self.model[x[:-1]][x[-1:]])
            if words[idx].isnumeric():
                continue
            words[idx] = max_prob[-1:][0]

        return words

In [35]:
blueberry = BlueberryCorrector(datafile=big_data_path,
                               ngram_dict_path=bigrams_path)
blueberry.load_nd()
blueberry.correct("I lyke moreniing cofee")
import time
t = time.time()
print(blueberry.correct("Blck wman lks coffe."))
print(blueberry.correct("Sugarr dady have a sex with madame."))
print(blueberry.correct("Do u like semen?"))
print(time.time() - t)

Black man los coffee.
Sugar daddy have a sex with madame.
Do u like semen?
0.0019080638885498047


In [36]:
class ErrorMaker:
    def __init__(self, spoil_prob: float=0.5):
        self.spoil_prob = spoil_prob # greater probability => more chances

    def spoil(self, sentence: str) -> str:
        tokens = re.findall(r"\w+", sentence)
        words = []
        for token in tokens:
            if random.random() < self.spoil_prob:
                if random.random() < 0.95:
                    words.append(
                        random.choice(tuple(self._edits1(token.lower())))         
                    )
                else:
                    words.append(
                        random.choice(tuple(self._edits2(token.lower())))         
                    )
            else:
                words.append(token)
        
        str_arr = []
        start_idx = 0
        next_start_idx = 0
        for idx, token in enumerate(tokens):
            # find index of current token
            next_start_idx = sentence.find(token, start_idx)
            # add everything between tokens also (e.g. punctuation)
            str_arr.append(sentence[start_idx:next_start_idx])
            to_add = words[idx]
            # capitalize if it was titled
            if token.istitle():
                to_add = to_add.capitalize()
            elif token.isupper():
                to_add = to_add.upper()
            str_arr.append(to_add)
            # set start index to the next symbol after the word
            start_idx = next_start_idx + len(token)
        # add the rest after the last token
        str_arr.append(sentence[start_idx:])

        return ''.join(str_arr)

    def _edits1(self, word: str) -> Set:
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def _edits2(self, word: str) -> Set: 
        "All edits that are two edits away from `word`."
        return set(e2 for e1 in self._edits1(word) for e2 in self._edits1(e1))

In [37]:
error_maker = ErrorMaker()
err = error_maker.spoil("I like my mom!")

In [38]:
blueberry.correct(err)

'Id like he mob!'

## Justify your decisions
In your implementation you will need to decide which ngram dataset to use, which weights to assign for edit1, edit2 or absent words probabilities, beam search parameters and etc. Write down justificaitons for these choices.

First of all, we had choice between bigrams and 5-grams dataset, cause there was a problem to obtain other. As for doing my own dataset I simply decided to use bigrams over the 5-grams, because it's not that sparse and could cover sentences of length two. Moreover, I used dictionary of words (unigrams) to cover single-word sentences.

As for weights it could pe possible to change probabilities if we obtain it from `edit1` or `edit2` functions, hence we should keep from where do we get this result. I've decided instead just use different tecnhiques to generate errors, mostly for the my tests I set up probability of the first type of error to be generated 95% more than the second one. Theoretically we also could give more weight to choosing probability taking into account that for bigrams should give better result because of context, but actually let me give example: if the first token in sentence is error of `edits2`, then bigrams could spoiled entire sentence, because errors we be accumulated. Whereas errors of `edits1` more tolerant, and yet in case of huge number of errors unigrams could give better results for my case.

Couple of words about absent words. If we couldn't find any possible unigram or bigram so we simply think it's just possible word, technically it's zero probability, but we don't mind, because we just yield it as it's the only option.

Beam Search interesting thing as I found here: https://towardsdatascience.com/foundations-of-nlp-explained-visually-beam-search-how-it-works-1586b9849a24, but I think it could also bumps into problem with accumulatig error, although to a lesser extent.


Additionally, I noticed where is the problem with contractions e.g., it would be different tokens, and if unigrams of bigrams dataset doesn't have any occurence of it, they will lead to wrong correction of sentence. We could improve model by using using trigrams, 4-grams, 5-grams, beam search, but it also demands on the good dataset with small number of errors and semantically true dependencies.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [39]:
# sents = nltk.sent_tokenize(nltk.corpus.treebank.raw()[:1000000])[:10]
sents = [
    "I checked to make sure that he was still alive.",
    "Many types occur just once in the LOB corpus.",
    "Compact discs are available which contain a corpus of a whole century of poetry on a single disc.",
    "Where could I find this fucking dataset with sentences?",
    "I advise EVERYONE DO NOT BE FOOLED!",
    "I bought this to use with my Kindle Fire and absolutely loved it!",
    "Revolution in Just Listening is the third studio album to be released by Missouri band Coalesce, which was released on November 16, 1999 through Relapse Records.",
]

spoileds = [error_maker.spoil(sent) for sent in sents]

blu_acc = blueberry.accuracy(sents, spoileds)
nor_acc = norvig.accuracy(sents, spoileds)

print(f"Blueberry's accuracy: {blu_acc:.2f}")
print(f"Norvig's accuracy: {nor_acc:.2f}")

Blueberry's accuracy: 0.87
Norvig's accuracy: 0.88


In [40]:
text = """I did not have many expectations going into this with my daughter, I've seen the main Shrek movies but noon of the other Puss in Boot's spin-offs. I was shocked at how good a movie this was when the credits rolled!
The story itself is functional, but all characters are given meaningful and relatable arcs. Between the melanchonic Puss reaching the end of his days as a great hero, his always positive dog sidekick and the 3 antagonistic parties trying to reach the magical macguffin there is a lot going on and none of it feels forced in.
The two greatest assets the movie brings is it's humor and the stunning action scenes. First of all, this movie has bite, it is back to the tone of the first Shrek movie with lots of jokes that are working on a children and adult level of understanding. While the story is easy to follow for kids, the themes explored are relatively mature and will keep adults engaged.
Secondly, while Disney/Pixar movies lately fail to make action exiting with their polished CGI style, Puss in boots goes full Into the Spiderverse once a fight breaks out. Glorious 12 frames per second, hyper stylized with all the filters and gimmicks necessary to elevate the big set-pieces to something truly special and memorable. Especially Puss's duels against a mysterious bounty hunter are the highlights.
While not entirely original, the Puss in Boots: The Last Wish combines the edgy humor of Shrek with the visual wonders of Into the Spiderverse and strings it along a relatively matured heroes journey coming to it's end tale that is closer to Logan than any other animation I can think off. Oh and just in case (not that I personally care much about it) there is zero political agenda to be found here to be a distraction of the perception.
"""
sents = nltk.sent_tokenize(text)

spoileds = [error_maker.spoil(sent) for sent in sents]
print(spoileds)

blu_acc = blueberry.accuracy(sents, spoileds)
nor_acc = norvig.accuracy(sents, spoileds)

print(f"Blueberry's accuracy: {blu_acc:.2f}")
print(f"Norvig's accuracy: {nor_acc:.2f}")

["I did noo have mdany expectatizons goinag into this hwith mym daughtter, Im've sveen tre maiq Shjek moaies but zoon of thej other Puss in Bnoot'g ssin-offs.", 'Ni was shockedj wt how bood a movie this was wlen the credits roluled!', 'The story itsoelf ig functionbal, bqt alla characters qare mgixven meaningful and relatable arcs.', 'Between the melanchonic Push reaching the end dof hzuis days as ga gteat hero, hpis aliays positive dog sidekick ad thb j3 antagooistic parties trying to reach thve magical macguffin mthere is ac loh going on anvd nond gof it fmeels forceg hn.', "Ieh twu greatoest assets the movie brings ias ivt'sb humor and thr stunning lction scenes.", 'First ocf all, thms mogvie has wbite, it ia backn gto the tone of tme first Shrlk movie with lots of jokes that ane working an a children and adult ledvel fo underszanding.', 'Whilre mhe storky cis easy to follzow fsor kihds, she themes oxplored lre relatilvely mature aed will kaep adults engafed.', 'Secondly, while Disn

In [41]:
text = """I did not expect the sequel to a decent spin-off Dreamworks film from over a decade ago to be one of the most poignant, introspective, genuinely hilarious, and heartwarming films of the year. But here we are.

After an overly cheesy, somewhat clunky opening sequence, The Last Wish very quickly begins developing its zany assortment of characters into distinct quirky personalities with sympathetic desires and clear goals. The film juggles several character arcs and it's almost miraculous how it successfully handled all of them with proper set up and satisfying, emotionally weighty payoffs.

The screenplay is wacky, witty, and also bursting with heart as it deals with weighty themes of trusting others and finding purpose in any circumstances. And it tackles these themes in ways that are always understandable to all ages but never insultingly oversimplified.

What I also didn't expect was that the action sequences would be so well-choreographed and beautifully animated, and that the movie would often be terrifying and violent at times.

I adored this film. I think it's Dreamworks' best film since Megamind and it's easily the best true family film of the year."""

sents = nltk.sent_tokenize(text)

spoileds = [error_maker.spoil(sent) for sent in sents]
print(spoileds)

blu_acc = blueberry.accuracy(sents, spoileds)
nor_acc = norvig.accuracy(sents, spoileds)

print(f"Blueberry's accuracy: {blu_acc:.2f}")
print(f"Norvig's accuracy: {nor_acc:.2f}")

['Fi dzid not exnect the sequeul to a decentn isppin-ofnf Dreamwrrks film from ovec va decade ago kto ba one of the most poignant, introspectivel, ogenuinely hilarious, and heartwarmiong films of theq year.', 'But here jrwe are.', 'After an overly cheesy, somewhat cluky openihng sequence, The Last Twish very quizckly begins developing its fzany assortment bf characters inio pistinct quirky persognalities witrh sympathetic desires and cpear goals.', "Theg fhilm juggles severax chxaracter arcs and ix'j almost miralulous how it successfully hansled ayl of xthem with proper szet um and satisfying, emaotionally weightyu payoffs.", 'Thme scrjenplay ies wacfky, wittcy, rnd alsuo bursting with hearp es itf deajls with weighty themes of trusting ethers abd finding purposen in any cirmcumstances.', 'Knd it tackles these themeks in ways that arde always understandable to all agbs bu nevev insultyngly oversimplified.', "Uhat Zi also disdn'f expect was tthat the action sequences woumd ze so well-ch

Sometimes better result keeps Blueberry, sometimes not, usually they differs for my implementation <5%, but it could be different, depends on sentences.

## Evaluate on Github Typo Corpus
Now, you need to evaluate your solution on the Github Typo Corpus. Don't forget to compare the accuracy with the Norvig's solution.

In [ ]:
!wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
!gzip -d github-typo-corpus.v1.0.0.jsonl.gz

In [ ]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] == 'eng' and edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((src, tgt))
                
print(f"Dataset size = {len(dataset)}")

Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [ ]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

Compare your implementation with Norvig's solution on this dataset

In [ ]:
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    # YOUR CODE HERE